In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import copy
import json

# Detectron colors
_COLORS = np.array([
    0.000, 0.447, 0.741,
    0.850, 0.325, 0.098,
    0.929, 0.694, 0.125,
    0.494, 0.184, 0.556,
    0.466, 0.674, 0.188
]).astype(np.float32).reshape((-1, 3))

# Random number generator seed
_RNG_SEED = 1

# Fix RNG seeds
random.seed(_RNG_SEED)
np.random.seed(_RNG_SEED)

# Directory where sweep summaries are stored
_DATA_DIR = '../data'

# Job summary entries to append (different across re-runs; e.g. job id)
_KEYS_TO_APPEND = ['job_id', 'rng_seed']

# Job summary entries to copy (same across re-runs; e.g. flops)
_KEYS_TO_CP = ['exp_mem', 'params', 'flops', 'net', 'optim']

# Job summary entries to average (likely different across re-runs; e.g. error)
_KEYS_TO_AVG = [
    'act_mem', 'prec_time', 'iter_time', 'min_test_top1',
    'train_ep_loss', 'train_ep_top1', 'test_ep_top1', 'train_it_loss', 'train_it_top1'
]

# Max flops constraint
_MAX_F = 0.129
# Max params constraint
_MAX_P = 0.856

In [ ]:
def load_sweep(sweep_name):
    """Loads a sweep summary."""
    summary_path = os.path.join(_DATA_DIR, '{}.json'.format(sweep_name))
    with open(summary_path, 'r') as f:
        sweep_summary = json.load(f)
    return sweep_summary


def avg(vals, decs=3):
    """Computes the average of a list of values."""
    if isinstance(vals[0], (int, float)):
        return round(np.mean(vals).item(), decs)
    if isinstance(vals[0], list):
        return np.around(np.mean(np.array(vals), axis=0), decs).tolist()
    if isinstance(vals[0], dict):
        return {k: avg([v[k] for v in vals]) for k in vals[0].keys()}
    raise NotImplementedError(type(vals[0]))


def combine_jobs(jobs):
    """Combines job summaries from different re-runs of the same job."""
    comb_job = {}
    # Append values that are different across re-runs (e.g. job id)
    for key in _KEYS_TO_APPEND:
        comb_job['{}s'.format(key)] = [job[key] for job in jobs]
    # Copy values that are shared across re-runs (e.g. flops)
    for key in _KEYS_TO_CP:
        comb_job[key] = copy.deepcopy(jobs[0][key])
    # Average values that are likely different across re-runs (e.g. error)
    for key in _KEYS_TO_AVG:
        comb_job[key] = avg([job[key] for job in jobs])
    return comb_job


def hash_job(job, w_bt=True):
    """Computes a hash for a job."""
    return json.dumps(
        {k: v for (k, v) in job['net'].items() if w_bt or k != 'block_type'},
        sort_keys=True
    )


def combine_sweeps(sweep_summaries):
    """Combines sweep summaries from multiple re-runs of the same sweep."""
    # Group job summaries by hash
    hash_to_js = {}
    for ss in sweep_summaries:
        for js in ss:
            js_hash = hash_job(js)
            if js_hash not in hash_to_js:
                hash_to_js[js_hash] = [js]
            else:
                hash_to_js[js_hash].append(js)
    # Combine job summaries by hash
    return [
        combine_jobs(job_summaries) for job_summaries in hash_to_js.values()
    ]

In [ ]:
# lr wd sweeps
sweeps_lr_wd = {
    'Vanilla': load_sweep('Vanilla_lr-wd'),
    'ResNet': load_sweep('ResNet_lr-wd'),
    'DARTS': load_sweep('DARTS_lr-wd')
}

In [ ]:
# Model rerun sweeps
sweeps_reruns = {
    'Vanilla': load_sweep('Vanilla_reruns'),
    'ResNet': load_sweep('ResNet_reruns')
}

In [ ]:
# RNG seed sweeps
sweeps_rng = {}
for ds in ['Vanilla', 'ResNet']:
    sweeps_rng[ds] = {}
    for rng in [1, 2, 3]:
        sweep_name = '{}_rng{}'.format(ds, rng)
        sweeps_rng[ds][rng] = load_sweep(sweep_name)

In [ ]:
# Average across seeds
sweeps_avg = {}
for ds in ['Vanilla', 'ResNet']:
    sweeps_avg[ds] = {}
    for num_runs in [1, 2, 3]:
        sweeps_avg[ds][num_runs] = combine_sweeps([
            sweeps_rng[ds][rng] for rng in range(1, num_runs + 1)
        ])

In [ ]:
print('Figure 12\n')

r, c = 1, 3
w, h = 4, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))

hps = ['base_lr', 'wd']
lbs = ['learning rate (log10)', 'weight decay (log10)']
dss = ['Vanilla', 'ResNet', 'DARTS']

def_pt = [0.1, 5e-4]
def_pt_log = np.log10(def_pt)

for j, ds in enumerate(dss):
    ax = axes[j] if r == 1 else axes[i, j]
    sweep = sweeps_lr_wd[ds]
    xs = [job['optim'][hps[0]] for job in sweep]
    ys = [job['optim'][hps[1]] for job in sweep]
    # Use log10 scale
    xs_log = np.log10(xs)
    ys_log = np.log10(ys)
    # Compute relative ranks
    errs = [job['min_test_top1'] for job in sweep]
    ranks = np.argsort(np.argsort(errs))
    ranks += 1
    ranks_rel = ranks / (len(ranks))
    # Plot relative ranks
    s = ax.scatter(xs_log, ys_log, c=ranks_rel, alpha=0.1, cmap='viridis', rasterized=True)
    ax.set_xlabel('{}'.format(lbs[0]), fontsize=16)
    if j == 0:
        ax.set_ylabel('{}'.format(lbs[1]), fontsize=16)
    xlim_log = np.log10([0.001, 1.0])
    ylim_log = np.log10([0.00001, 0.01])
    ax.set_xlim(xlim_log)
    ax.set_ylim(ylim_log)
    ax.grid(alpha=0.4)
    # Show default setting
    def_pt_alpha = 0.8
    pr_col = _COLORS[1]
    ax.scatter(def_pt_log[0], def_pt_log[1], color=pr_col, alpha=def_pt_alpha)
    ax.plot(
        np.linspace(xlim_log[0], def_pt_log[0], 10), [def_pt_log[1] for _ in range(10)],
        color=pr_col, alpha=def_pt_alpha, linestyle='--', linewidth=2.5
    )
    ax.plot(
        [def_pt_log[0] for _ in range(10)], np.linspace(ylim_log[0], def_pt_log[1], 10),
        color=pr_col, alpha=def_pt_alpha, linestyle='--', linewidth=2.5
    )
    ax.set_title(ds, fontsize=16)

fig.colorbar(s, ax=axes.ravel().tolist());

In [ ]:
print('Figure 13\n')

r, c = 1, 2
w, h = 4, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))

dss = ['Vanilla', 'ResNet']
cs = ['top', 'mid']
cs_ls = ['top-ranked', 'mid-ranked']
bin_size = 0.1

for i, ds in enumerate(dss):
    ax = axes[i]
    for j, c in enumerate(cs):
        sweep = sweeps_reruns[ds][c]
        errs = [job['min_test_top1'] for job in sweep]
        num_bins = int((max(errs) - min(errs)) / bin_size)
        ax.hist(
            errs, bins=num_bins, color=_COLORS[j], alpha=0.8,
            label='{}'.format(cs_ls[j])
        )
    if ds == 'Vanilla':
        ax.set_xlim([4.5, 10])
    else:
        ax.set_xlim([4.5, 10])
    ax.grid(alpha=0.4)
    ax.set_ylim([0, 40])
    ax.legend(loc='upper right', prop={'size': 13}, ncol=1)
    ax.set_xlabel('error', fontsize=16)
    ax.set_ylabel('number of runs', fontsize=16)
    ax.set_title(ds, fontsize=16)

plt.tight_layout();

In [ ]:
print('Figure 14\n')

# Plot EDFs for varying number of reruns
r, c = 1, 2
w, h = 4, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))
dss = ['Vanilla', 'ResNet']

for i, ds in enumerate(dss):
    ax = axes[i]
    for j in [1, 2, 3]:
        sweep = [job for job in sweeps_avg[ds][j]]
        errs = sorted([job['min_test_top1'] for job in sweep])
        ax.plot(
            errs, np.linspace(0, 1, len(errs)),
            color=_COLORS[j], linewidth=2, alpha=0.8, label='{} reruns'.format(j)
        )
    ax.grid(alpha=0.4)
    ax.set_xlabel('error', fontsize=16)
    ax.set_ylabel('cumulative prob.', fontsize=16)
    ax.set_xlim([4.5, 17.5])
    ax.set_xticks([5.0, 7.5, 10.0, 12.5, 15.0, 17.5])
    ax.set_title(ds, fontsize=16)
    ax.legend(loc='lower right', prop={'size': 13})

plt.tight_layout();